# Setup

In [1]:
from os import environ

environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/lexaux/work/freespeech/id/alex-prod-access-freespeech-343914-3b04c8fb86f9.json"
environ["DEEPGRAM_TOKEN"] = "853231a8db620681622baaef76fab33070613d19"
environ["NOTION_TOKEN"] = "secret_uLJMnhu0cXfPq2vkkIIVg0OjmytdWscQiN9nOQbTcVP"
environ["AZURE_SUBSCRIPTION_KEY"] = "5e872cb4b1694ca1aa785035dc64817e"
environ["AZURE_REGION"] = "eastus"

OUTPUT_DIR = "/home/lexaux/work/freespeech/output"

# Download Video

A three-minute English teaching dialogue of ladies discussing the

In [2]:
from freespeech.lib import youtube


VIDEO_URL = "https://youtu.be/VyRE4bgrNDs"
audio_file, video_file, meta, subtitles = youtube.download(VIDEO_URL, output_dir=OUTPUT_DIR)

# Cut Video

In [3]:
from freespeech.lib import media


START = '00:00:00'
FINISH = '00:02:00'

cut_audio_file = await media.cut(audio_file, start=START, finish=FINISH, output_dir=OUTPUT_DIR)
cut_video_file = await media.cut(video_file, start=START, finish=FINISH, output_dir=OUTPUT_DIR)

cut_video_with_audio_file = await media.dub(cut_video_file, cut_audio_file, output_dir=OUTPUT_DIR)

# Transcribe

This would be an english-to-english dub so that we can see how well it fits against the original phrases and tempo.
No need to translate here.

In [4]:
from freespeech.lib import media, speech
from freespeech.lib.storage import obj

audio_file = cut_audio_file

audio_file_mono = await media.multi_channel_audio_to_mono(audio_file, output_dir=OUTPUT_DIR)
(audio_info, *_), _ = media.probe(audio_file_mono)

audio_uri = f"gs://freespeech-tests/transcribe/{audio_file_mono.name}"
await obj.put(audio_file_mono, audio_uri)

transcript = await speech.transcribe(audio_uri, audio_info, "en-US", "default", provider="Deepgram")

# Normalize

Normalizing the breaks and also ensuring voice is Azure.

In [6]:
from freespeech.types import Voice
from dataclasses import replace

normalized_events = speech.normalize_speech(transcript, gap_ms=2000, length=500, method="break_ends_sentence")

# Deepgram gives us only 'Alonzo Church' and 'Alan Turing' voices after recognition.
# We want Melinda in this case, so...
events = [replace(e, voice=Voice(character="Melinda", pitch=0.0, speech_rate=None)) for e in normalized_events]

# Text to speech with Azure

In [7]:
synth_output, voices = await speech.synthesize_events(events, "Melinda", "en-US", 0, OUTPUT_DIR)
mixed_file = await media.mix((synth_output, audio_file), (1, 0.4),  output_dir=OUTPUT_DIR)
dubbed_video = await media.dub(cut_video_file, mixed_file, output_dir=OUTPUT_DIR)

dubbed_video

retrying delta=998 rate=1.0
retrying delta=8122 rate=1.0
retrying delta=310 rate=1.2815251299826689
retrying delta=1867 rate=1.0
retrying delta=6013 rate=1.0
retrying delta=1617 rate=1.0
retrying delta=5754 rate=1.0
retrying delta=2997 rate=1.0
retrying delta=5876 rate=1.0


PosixPath('/home/lexaux/work/freespeech/output/eabcdd83-7dc3-451e-99aa-5e45da52c878.mp4')